In [ ]:
import os
import json
from pathlib import Path

from gretel_client.projects import get_project, search_projects, models, Project

In [ ]:
PRE_TRAIN_PROJECT = "gretel-client-project-pretrained"

FIXTURES = Path().resolve()

FIXTURES


In [ ]:
project = get_project(name=PRE_TRAIN_PROJECT)

project.name

In [ ]:
def poll(model):
    for update in model.poll_logs_status():
        if update.transitioned:
            print(f"Status is {update.status}")
        for log in update.logs:
            msg = f"{log['ts']}  {log['msg']}"
            if log["ctx"]:
                msg += f"\n{json.dumps(log['ctx'], indent=4)}"
            print(msg)
        if update.error:
            print(f"\t{update.error}")

In [ ]:
def build_synth_model(project: Project):
    model = project.create_model_obj(model_config="synthetics/default")
    model.data_source = str(FIXTURES / "us-adult-income.csv")
    model.submit(runner_mode=models.RunnerMode.CLOUD, upload_data_source=True)
    poll(model)
    return model

syn_model = build_synth_model(project)  # 626ab963f7098ebecbb623b2
syn_model.model_id

In [ ]:
def build_xf_model(project: Project):
    model = project.create_model_obj(model_config=FIXTURES / "transforms_config.yml")
    model.data_source = str(FIXTURES / "account-balances.csv")
    model.submit(runner_mode=models.RunnerMode.CLOUD, upload_data_source=True)
    poll(model)
    return model

xf_model = build_xf_model(project)  # 626abd19f7098ebecbb623d9
xf_model.model_id

In [ ]:
def build_classify_model(project: Project):
    model = project.create_model_obj(model_config=FIXTURES / "classify_config.yml")
    model.data_source = str(FIXTURES / "account-balances.csv")
    model.submit(runner_mode=models.RunnerMode.CLOUD, upload_data_source=True)
    poll(model)
    return model

classify_model = build_classify_model(project)  # 626abd2ff7098ebecbb623da
classify_model.model_id